In [3]:
import pandas as pd
from keras import Sequential
from keras import layers
from keras import regularizers
import numpy as np
from string import punctuation
import pyprind
from collections import Counter
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os

# for reproducability
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)


def read_array(fname):
    '''read an array from txt file'''
    with open(fname) as f:
        for line in f:
            pairs = line.split(',')
            #print(pairs)
    return [float(item) for item in pairs]

## Load training data

In [4]:
# define paths of the Aesthetic Features
path_AF_Mean = '/media/stone/Data/DataSet_me18me/me18me-devset/dev-set/features/aesthetic_visual_features/aesthetic_feat_dev-set_mean'
path_AF_Median = '/media/stone/Data/DataSet_me18me/me18me-devset/dev-set/features/aesthetic_visual_features/aesthetic_feat_dev-set_median'

# Load video related features first
vn_mean = os.listdir(path_AF_Mean)

# stack the video names in dataframe
df = pd.DataFrame()
df['video'] = [os.path.splitext(vn)[0]+'.webm' for vn in vn_mean]

# read the aesthetic feat (mean and media) in dataframe
df['AF_mean'] = [ read_array(path_AF_Mean+'/'+vn[:-5]+'.txt') for vn in df['video']]
df['AF_median'] = [ read_array(path_AF_Median+'/'+vn[:-5]+'.txt') for vn in df['video']]
df.head()

,video,AF_mean,AF_median
0,video10.webm,"[0.402878, 0.389358, 0.564687, 0.400378, 0.471...","[0.427258, 0.381448, 0.582746, 0.409475, 0.489..."
1,video100.webm,"[0.160705, 0.237773, 0.409616, 0.179789, 0.367...","[0.160579, 0.238155, 0.408475, 0.180416, 0.366..."
2,video10000.webm,"[0.46136, 0.358996, 0.710791, 0.333263, 0.5849...","[0.461391, 0.358922, 0.710828, 0.333437, 0.585..."
3,video10001.webm,"[0.105545, 0.60667, 0.569378, 0.515591, 0.4109...","[0.105085, 0.606699, 0.567169, 0.51536, 0.4101..."
4,video10002.webm,"[0.233127, 0.149484, 0.649683, 0.14423, 0.6057...","[0.232971, 0.149975, 0.649477, 0.143106, 0.605..."


In [5]:
# load the ground truth values
label_path = '/media/stone/Data/DataSet_me18me/me18me-devset/dev-set/ground-truth/'
labels=pd.read_csv(label_path+'ground-truth_dev-set.csv')
labels.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video10.webm,0.950,34,0.900,10
1,video100.webm,0.951,33,0.889,9
2,video10000.webm,0.832,33,1.000,13
3,video10001.webm,0.865,33,0.727,11
4,video10002.webm,0.899,59,0.792,24


In [6]:
# Align the labels with features according to name
df_complete = pd.merge(df,labels,on='video')
df_complete.head()

,video,AF_mean,AF_median,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video10.webm,"[0.402878, 0.389358, 0.564687, 0.400378, 0.471...","[0.427258, 0.381448, 0.582746, 0.409475, 0.489...",0.950,34,0.900,10
1,video100.webm,"[0.160705, 0.237773, 0.409616, 0.179789, 0.367...","[0.160579, 0.238155, 0.408475, 0.180416, 0.366...",0.951,33,0.889,9
2,video10000.webm,"[0.46136, 0.358996, 0.710791, 0.333263, 0.5849...","[0.461391, 0.358922, 0.710828, 0.333437, 0.585...",0.832,33,1.000,13
3,video10001.webm,"[0.105545, 0.60667, 0.569378, 0.515591, 0.4109...","[0.105085, 0.606699, 0.567169, 0.51536, 0.4101...",0.865,33,0.727,11
4,video10002.webm,"[0.233127, 0.149484, 0.649683, 0.14423, 0.6057...","[0.232971, 0.149975, 0.649477, 0.143106, 0.605...",0.899,59,0.792,24


## Split data

In [7]:
# use the AF_mean only
X =np.array([col for col in df_complete['AF_median'].values ])
Y = df_complete[['short-term_memorability','long-term_memorability']].values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=124)
X

array([[ 0.427258,  0.381448,  0.582746, ..., -0.27963 , -0.012414,
         0.197281],
       [ 0.160579,  0.238155,  0.408475, ...,  0.719529,  1.099627,
         1.690907],
       [ 0.461391,  0.358922,  0.710828, ...,  1.56886 ,  0.38056 ,
        -1.09876 ],
       ...,
       [ 0.358854,  0.134898,  0.513471, ...,  1.503436,  0.299821,
         0.341942],
       [ 0.071911,  0.516086,  0.635375, ..., -0.199719,  0.394017,
        -0.115456],
       [ 0.560848,  0.06117 ,  0.627021, ...,  0.09347 ,  0.026843,
         0.421589]])

## Build SVR model

In [8]:
# building a pipeline
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.decomposition import KernelPCA, PCA
from sklearn.preprocessing import StandardScaler

In [9]:
def svr_pipline():
    stdi = StandardScaler()
    pca = PCA(n_components=0.98,svd_solver='full')
    svr = svm.SVR(C=1.0,epsilon=0.01,kernel='rbf')
    svr_pip = Pipeline([('standadizer',stdi),('pca',pca),('svr',svr)])
    return svr_pip

# predict the short-term and long-term memorability
MP_svr_s = svr_pipline()
MP_svr_l = svr_pipline()
MP_svr_s.fit(X_train,Y_train[:,0])
Y_pred_short=MP_svr_s.predict(X_test)

MP_svr_l.fit(X_train,Y_train[:,1])
Y_pred_long = MP_svr_l.predict(X_test)

In [10]:
# collect results and calculate the score using Spearman's rank correlation
res = pd.DataFrame()
res['short_pred'] = Y_pred_short
res['long_pred'] = Y_pred_long
res['short_true'] = Y_test[:,0]
res['long_true'] = Y_test[:,1]

In [11]:
res[['short_pred','short_true']].corr(method='spearman',min_periods=1)

,short_pred,short_true
short_pred,1.00000,0.23782
short_true,0.23782,1.00000


In [12]:
a=res[['long_pred','long_true']].corr(method='spearman',min_periods=1)
a.iloc[0,1]

0.0637144478361168

In [13]:
def spearman_corr(x_pred,x_true):
    "The official performance matrix: Spearman's rank correlation"
    a = pd.DataFrame()
    a['true'] = x_true
    a['pred'] = x_pred
    res = a[['true','pred']].corr(method='spearman',min_periods=1)
    return res.iloc[0,1]

from sklearn.metrics import make_scorer
spearman = make_scorer(spearman_corr,greater_is_better=True)

In [14]:
spearman_corr(Y_pred_short,Y_test[:,0])

0.2378199906344876

# Model Selection

In [15]:
from sklearn.model_selection import GridSearchCV

In [29]:
params = {'pca__n_components':[0.7,0.8,0.9,0.95],
         'svr__C':[0.01,0.1,1,2,5],
         'svr__epsilon':[0.01,0.1,0.2],
         'svr__kernel':['rbf','poly']}
A = GridSearchCV(MP_svr_s,params,scoring=spearman,cv=3,n_jobs=-1)

In [30]:
A.fit(X,Y[:,0])

Process ForkPoolWorker-68:
Process ForkPoolWorker-67:
Process ForkPoolWorker-69:
Process ForkPoolWorker-70:
Process ForkPoolWorker-71:


KeyboardInterrupt: 

In [ ]:
B = GridSearchCV(MP_svr_l,params,scoring=spearman,cv=3,n_jobs=-1)
B.fit(X,Y[:,1])

In [ ]:
# save results
# To serialize
import pickle
with open('svr_aesthetic_short.pkl', 'wb') as fid:
    pickle.dump(A, fid)
with open('svr_aesthetic_long.pkl','wb') as fid:
    pickle.dump(B,fid)
# To deserialize estimator later
#with open('our_estimator.pkl', 'rb') as fid:
#    gnb = pickle.load(fid)

In [ ]:
A.best_params_

In [ ]:
B.best_score_

In [ ]:
A.best_score_